In [1]:
# Load in necessary packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
# Based of EDA from previous notebook
X_cols = ["B_2", "B_7", "B_18", "B_23", "B_32", "D_48", "D_55", "D_61", "D_121", "P_2", "S_11"]

In [3]:
# Load in the data
chunksize = 1000000

train_df_iter = pd.read_csv('train_data.csv', chunksize = chunksize, usecols = ["customer_ID"] + X_cols)
train_df_labels = pd.read_csv("train_labels.csv")

train_df = pd.DataFrame()

for i_chunk, chunk in enumerate(train_df_iter):
    train_df = pd.concat([train_df, chunk])
    print(train_df.shape)
    
# Group the data by the "customer_ID"
train_df_mean = train_df.groupby("customer_ID")[X_cols].mean().reset_index()
train_df_last = train_df.groupby("customer_ID")[X_cols].last().reset_index()

# Merge the mean and the last into one dataset
train_df = pd.merge(
    left=train_df_mean, 
    right=train_df_last, 
    how="inner",
    on="customer_ID",
    suffixes=("_mean", "_last"),
)

# Merge labels and data
train_df = pd.merge(train_df, train_df_labels, on = "customer_ID", how = "left")

train_df.head()

(1000000, 12)
(2000000, 12)
(3000000, 12)
(4000000, 12)
(5000000, 12)
(5531451, 12)


,customer_ID,B_2_mean,B_7_mean,B_18_mean,B_23_mean,B_32_mean,D_48_mean,D_55_mean,D_61_mean,D_121_mean,...,B_18_last,B_23_last,B_32_last,D_48_last,D_55_last,D_61_last,D_121_last,P_2_last,S_11_last,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.005086,0.036624,0.842565,0.026177,0.005084,0.240978,0.224432,0.225847,0.711829,...,1.007897,0.040367,0.007645,0.192376,0.187285,0.227637,0.719791,0.934745,0.402246,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.991083,0.028049,1.004884,0.013286,0.005960,0.048203,0.048069,0.053319,0.535892,...,1.003602,0.014705,0.008645,0.014696,0.036112,0.048978,0.551341,0.880519,0.363754,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.815677,0.034433,0.933173,0.023436,0.005220,0.092284,0.077362,0.109526,0.431903,...,1.004080,0.020228,0.001252,0.080370,0.098963,0.137834,0.444615,0.880875,0.280417,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.955264,0.062130,0.910999,0.046339,0.005252,0.076686,0.061726,0.066872,0.621386,...,1.007289,0.005060,0.007541,0.013057,0.021400,0.026844,0.629147,0.621776,0.368774,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.814543,0.115290,0.610194,0.100755,0.005332,0.253697,0.203298,0.356445,0.550940,...,0.531486,0.145214,0.007263,0.325121,0.254067,0.600739,0.565815,0.871900,0.326776,0


In [4]:
train_df["D_55_last"] = train_df[["D_55_last"]].fillna(train_df[["D_55_last"]].mean())
train_df["D_55_mean"] = train_df[["D_55_mean"]].fillna(train_df[["D_55_mean"]].mean())

train_df["P_2_mean"] = train_df[["P_2_mean"]].fillna(train_df[["P_2_mean"]].mean())
train_df["P_2_last"] = train_df[["P_2_last"]].fillna(train_df[["P_2_last"]].mean())

train_df["B_2_mean"] = train_df[["B_2_mean"]].fillna(train_df[["B_2_mean"]].mean())
train_df["B_2_last"] = train_df[["B_2_last"]].fillna(train_df[["B_2_last"]].mean())

train_df["D_48_mean"] = train_df[["D_48_mean"]].fillna(train_df[["D_48_mean"]].mean())
train_df["D_48_last"] = train_df[["D_48_last"]].fillna(train_df[["D_48_last"]].mean())

train_df["D_61_mean"] = train_df[["D_61_mean"]].fillna(train_df[["D_61_mean"]].mean())
train_df["D_61_last"] = train_df[["D_61_last"]].fillna(train_df[["D_61_last"]].mean())

train_df["D_121_mean"] = train_df[["D_121_mean"]].fillna(train_df[["D_121_mean"]].mean())
train_df["D_121_last"] = train_df[["D_121_last"]].fillna(train_df[["D_121_last"]].mean())

In [5]:
_X_cols = train_df.columns[1:-1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df[_X_cols], train_df["target"], 
                                                   test_size = 0.2, random_state = 42, stratify = train_df["target"])

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((367130, 22), (91783, 22), (367130,), (91783,))

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

model_5 = tf.keras.Sequential([
    # tf.keras.layers.Input(shape = (None, 22)),
    Dense(128, activation = 'relu', name = "dense_1"),
    Dense(64, activation = 'relu', name = "dense_2"),
    Dense(32, activation = 'relu', name = "dense_3"),
    Dense(16, activation = 'relu', name = "dense_4"),
    Dense(1, activation = 'sigmoid', name = "dense_5")
])

model_5.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["accuracy"])

2022-06-13 14:48:28.993795: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model_5.fit(X_train, y_train, epochs = 2)

Epoch 1/2
11473/11473 [==============================] - 15s 1ms/step - loss: 0.2795 - accuracy: 0.8722
Epoch 2/2
11473/11473 [==============================] - 14s 1ms/step - loss: 0.2736 - accuracy: 0.8746


In [9]:
chunksize = 1000000

test_df_iter = pd.read_csv("test_data.csv", chunksize=chunksize, usecols=["customer_ID"] + X_cols)

In [10]:
_X_cols = train_df.columns[1:-1]

_index = []
_vals = []

for chunk in test_df_iter:
    _chunk_mean = chunk.groupby("customer_ID")[X_cols].mean().reset_index()
    _chunk_last = chunk.groupby("customer_ID")[X_cols].last().reset_index()
    _chunk = pd.merge(
        left=_chunk_mean, 
        right=_chunk_last, 
        how="inner",
        on="customer_ID",
        suffixes=("_mean", "_last"),
    )

    X_test = _chunk[_X_cols]
    # print(X_test.shape)
    X_test = X_test.fillna(X_test.mean())
    y_test_pred = model_5.predict(X_test)
    y_test_pred = y_test_pred.squeeze()#[:,1]
    # y_test_pred = y_test_pred.reshape(1,-1)
    # y_test_pred = y_test_pred[:,1]
    _index.extend(_chunk["customer_ID"])
    _vals.extend(y_test_pred)
    
    print(len(_index))

81358
162618
244059
325450
406815
488260
569555
650904
732217
813543
895040
924631


In [11]:
res_df = pd.DataFrame(
    {"customer_ID": _index, "prediction": _vals}
).groupby("customer_ID")["prediction"].mean().reset_index()

In [13]:
res_df

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.088062
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001656
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.034580
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.327925
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.786373
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.292073
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.704008
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.419216
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.193813


In [14]:
res_df.to_csv("tensorflow_model_submission1.csv", index = False)